In [1]:
import os
import numpy as np
import pandas as pd
import re

from utils.logger import LOGPATH

In [2]:
fpath = LOGPATH.joinpath('summary-fb.csv')
tori = pd.read_csv(fpath, engine='python')
tori

,seed,data,model,conv,epoch_best,epoch_learn,time_learn,mem_ram_train,mem_cuda_train,f1macro_train,f1micro_train,f1macro_val,f1micro_val,f1macro_test,f1micro_test,time_eval,mem_ram_eval,mem_cuda_eval
0,20,cora,Iterative,AdjConv,63,500,4.1765,1.080,0.059,92.248,93.169,88.792,90.037,87.530,89.094,0.0011,1.080,0.059
1,21,cora,Iterative,AdjConv,45,500,4.0485,1.128,0.059,75.728,85.231,73.496,83.764,72.777,81.331,0.0011,1.128,0.059
2,22,cora,Iterative,AdjConv,50,500,4.0349,1.132,0.059,76.519,85.908,70.762,79.705,76.251,86.876,0.0011,1.132,0.059
3,20,cora,Iterative,ChebConv,47,500,6.4711,1.100,0.071,99.386,99.508,69.611,73.432,73.348,75.416,0.0017,1.100,0.071
4,21,cora,Iterative,ChebConv,46,500,6.2731,1.147,0.071,98.865,99.077,72.122,75.461,73.184,75.231,0.0017,1.147,0.071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,21,penn94,DecoupledFixedCompose,"Adji2Conv,Adji2Conv-gaussian,gaussian",287,500,107.7445,3.517,1.861,81.109,81.124,77.627,77.638,76.942,76.981,0.0092,3.517,1.861
896,22,penn94,DecoupledFixedCompose,"Adji2Conv,Adji2Conv-gaussian,gaussian",37,500,106.8353,3.517,1.861,67.480,67.547,65.441,65.542,66.203,66.289,0.0093,3.517,1.861
897,20,penn94,DecoupledFixedCompose,"AdjDiffConv,AdjDiffConv-appr,appr",111,500,58.1150,3.006,1.393,86.051,86.178,75.013,75.216,75.237,75.564,0.0093,3.006,1.393
898,21,penn94,DecoupledFixedCompose,"AdjDiffConv,AdjDiffConv-appr,appr",107,500,55.9517,3.508,1.393,85.290,85.401,75.421,75.589,74.585,74.907,0.0085,3.508,1.393


In [3]:
dlst = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire", \
        "amazon_ratings", "minesweeper", "tolokers", "questions", "reddit", "penn94"]
id_cols = ["seed", 'data', 'model', 'conv',]
m_cols = ['epoch_learn', 'time_learn', 'mem_ram_train', 'mem_cuda_train', 'f1micro_test', 'time_eval',]
t = tori[tori['data'].isin(dlst)][id_cols+m_cols].sort_values(by=['data', 'model', 'conv', 'seed'])
t.sort_index(ascending=False).groupby(['data', 'model', 'conv']).filter(lambda x: len(x) > 3)

,seed,data,model,conv,epoch_learn,time_learn,mem_ram_train,mem_cuda_train,f1micro_test,time_eval


In [4]:
conv_repr = {
    'MLP': {
        'Identity': 'Identity',
    },
    'DecoupledFixed': {
        'AdjConv-impulse':  'Impulse',
        'AdjConv-mono':     'Monomial',
        'AdjConv-appr':     'PPR',
        'AdjConv-hk':       'HK',
        'AdjConv-gaussian': 'Gaussian',
    },
    'DecoupledVar': {
        'AdjConv':          'Var-Monomial',
        'HornerConv':       'Horner',
        'ChebConv':         'Chebyshev',
        'ChebConv2':        'ChebInterp',
        'ClenhawConv':      'Clenshaw',
        'BernConv':         'Bernstein',
    },
    'AdaGNN': {'AdaConv':   'AdaGNN'},
    'ACMGNN': {
        # 'ACMConv-1.0-low-high':     'FBGNNI',
        # 'ACMConv-2.0-low-high':     'FBGNNII',
        # 'ACMConv-1.0-low-high-id':  'ACMGNNI',
        # 'ACMConv-2.0-low-high-id':  'ACMGNNII',
        'ACMConv-2.0-low-high-id':  'ACMGNN',
    },
    'DecoupledFixedCompose': {
        'AdjiConv,AdjiConv-mono,mono': 'FAGNN',
        'Adji2Conv,Adji2Conv-gaussian,gaussian': 'G$^2$CN',
        'AdjDiffConv,AdjDiffConv-appr,appr': 'GNN-LF/HF',
    },
    'DecoupledVarCompose': {
        'AdjConv,ChebConv,BernConv': 'FiGURe',
    }
}
flst = []
for m in conv_repr.values():
    for c, name in m.items():
        flst.append(name)

df = t.groupby(['data', 'model', 'conv'])[m_cols].agg(['mean', 'std'])
df.columns = ['_'.join(col).strip() for col in df.columns.values]
df['time_learn_mean'] = df['time_learn_mean'] * 1000 / df['epoch_learn_mean']
df['time_learn_std'] = df['time_learn_std'] * 1000 / df['epoch_learn_mean']
df['time_eval_mean'] = df['time_eval_mean'] * 1000
df['time_eval_std'] = df['time_eval_std'] * 1000
df.drop(columns=['epoch_learn_mean', 'epoch_learn_std'], inplace=True)
df.reset_index(inplace=True)

df['name'] = df.apply(lambda row: conv_repr[row['model']][row['conv']] if (row['model'] in conv_repr) and (row['conv'] in conv_repr[row['model']]) else None, axis=1)
df.dropna(subset=['name'], inplace=True)
df.drop(columns=['model', 'conv'], inplace=True)
df = df[['data', 'name'] + df.columns.tolist()[1:-1]]
df

,data,name,time_learn_mean,time_learn_std,mem_ram_train_mean,mem_ram_train_std,mem_cuda_train_mean,mem_cuda_train_std,f1micro_test_mean,f1micro_test_std,time_eval_mean,time_eval_std
3,actor,ACMGNN,42.606200,1.100886,1.230000,0.058847,0.413000,0.000000,32.675667,2.957597,7.400000,0.264575
4,actor,AdaGNN,10.009533,1.578104,1.170667,0.049075,0.141000,0.000000,38.070333,0.833196,1.466667,0.115470
5,actor,PPR,9.703267,0.443809,1.130667,0.049075,0.108000,0.000000,37.236667,0.840125,1.300000,0.173205
6,actor,Gaussian,9.855933,1.504255,1.151667,0.057274,0.109000,0.000000,37.763333,0.174053,1.400000,0.200000
7,actor,HK,9.421800,1.131418,1.120000,0.050269,0.108000,0.000000,35.679667,0.665349,1.100000,0.173205
...,...,...,...,...,...,...,...,...,...,...,...,...
285,tolokers,ChebInterp,59.358267,1.791183,1.199667,0.069400,0.235667,0.000577,79.129333,0.673138,9.633333,1.365040
286,tolokers,Clenhaw,25.748867,2.445461,1.215333,0.071143,0.234000,0.000000,78.576667,0.780269,3.166667,0.057735
287,tolokers,Horner,27.344933,2.159201,1.192667,0.062164,0.228667,0.000577,78.236333,0.619582,2.966667,0.152753
288,tolokers,FiGURe,169.319467,5.167309,1.213667,0.069759,1.082000,0.000000,78.137000,0.449778,43.800000,0.435890


## Latex - Acc

In [107]:
def calc_ranks(mean, std, ascending=False, exc=[]):
    rank = pd.Series(np.nan, index=mean.index)
    na_indices = mean[mean.isna()].index
    nrows = len(mean)
    rank[na_indices] = nrows

    mean = mean.dropna()
    mean = mean[~mean.index.isin(exc)]
    std = std.loc[mean.index]
    idx_pend = mean.sort_values(ascending=ascending).index
    current_rank = 1
    while not idx_pend.empty and current_rank <= nrows:
        idx_top = idx_pend[0]
        top_mean = mean[idx_top]
        top_std = std[idx_top]
        rank[idx_top] = current_rank

        if ascending:
            idx_same = idx_pend[mean[idx_pend] <= top_mean + top_std]
        else:
            idx_same = idx_pend[mean[idx_pend] >= top_mean - top_std]
        rank[idx_same] = current_rank

        idx_pend = idx_pend.difference(idx_same)
        idx_pend = mean.loc[idx_pend].sort_values(ascending=ascending).index
        current_rank += len(idx_same)

    return rank

rk = ['a', 'b', 'c']

In [36]:
dlst_acc = dlst
# dlst_acc = ["cora", "citeseer", "pubmed", "flickr", "chameleon_filtered", "squirrel_filtered", "actor", "roman_empire"]
acc_str = pd.DataFrame(index=flst, columns=dlst_acc, dtype=str)
for row in acc_str.index:
    for col in acc_str.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == col), 'f1micro_test_mean'].values
        std = df.loc[(df['name'] == row) & (df['data'] == col), 'f1micro_test_std'].values
        if len(mean) > 0:
            acc_str.loc[row, col] = f'{mean[0]:.2f}\\tpm{{{std[0]:.2f}}}'

# acc_str[acc_str.isna().any(axis=1)]
# acc_str.dropna(inplace=True)

for col in acc_str.columns:
    mean = df.loc[df['data'] == col, ['name', 'f1micro_test_mean']].set_index('name')['f1micro_test_mean'].astype(float)
    std  = df.loc[df['data'] == col, ['name', 'f1micro_test_std']].set_index('name')['f1micro_test_std'].astype(float)
    # rank = calc_ranks(mean, std)
    rank = mean.rank(ascending=False).astype(int)
    for i, rki in enumerate(rk):
        idx = rank[rank == i+1].index
        acc_str.loc[idx, col] = f'\\rk{rki}' + '{' + acc_str.loc[idx, col] + '}'

acc_str

,cora,citeseer,pubmed,flickr,chameleon_filtered,squirrel_filtered,actor,roman_empire,amazon_ratings,minesweeper,tolokers,questions,reddit,penn94
Identity,75.17\tpm{1.36},72.93\tpm{0.35},87.81\tpm{0.37},35.46\tpm{0.17},30.52\tpm{1.81},32.13\tpm{4.23},37.02\tpm{0.50},65.30\tpm{0.64},43.29\tpm{1.21},80.53\tpm{0.87},78.32\tpm{0.58},96.93\tpm{0.17},36.91\tpm{0.91},74.61\tpm{0.50}
Impulse,86.20\tpm{1.55},75.27\tpm{0.26},83.12\tpm{0.83},23.49\tpm{3.96},36.52\tpm{3.93},34.83\tpm{1.50},25.44\tpm{0.72},28.22\tpm{0.80},43.34\tpm{0.51},80.20\tpm{0.70},78.73\tpm{0.59},96.96\tpm{0.03},23.71\tpm{3.60},56.61\tpm{0.70}
Monomial,\rkb{88.85\tpm{1.19}},77.00\tpm{0.98},89.37\tpm{0.34},\rkb{37.41\tpm{0.63}},32.40\tpm{2.53},34.99\tpm{3.12},28.18\tpm{8.12},64.85\tpm{0.89},41.64\tpm{0.50},77.28\tpm{5.05},78.70\tpm{0.90},96.97\tpm{0.06},36.45\tpm{0.86},75.85\tpm{0.30}
PPR,\rka{89.09\tpm{1.82}},77.10\tpm{0.46},88.94\tpm{0.52},36.62\tpm{0.44},35.21\tpm{2.34},35.89\tpm{3.94},37.24\tpm{0.84},65.48\tpm{1.26},\rkc{43.97\tpm{0.81}},80.53\tpm{0.87},78.25\tpm{0.60},94.35\tpm{4.48},36.03\tpm{0.82},74.97\tpm{0.17}
HK,86.81\tpm{3.89},77.15\tpm{1.04},89.34\tpm{0.11},36.18\tpm{1.03},32.02\tpm{1.49},32.96\tpm{1.69},35.68\tpm{0.67},64.64\tpm{0.24},41.98\tpm{0.21},80.50\tpm{0.74},\rkb{78.92\tpm{0.79}},96.95\tpm{0.06},35.68\tpm{0.70},63.48\tpm{10.13}
Gaussian,88.36\tpm{1.78},77.15\tpm{0.89},89.03\tpm{0.26},35.64\tpm{1.62},35.21\tpm{1.30},36.19\tpm{1.97},37.76\tpm{0.17},\rkc{66.41\tpm{1.00}},43.38\tpm{1.14},81.00\tpm{1.61},78.34\tpm{0.59},\rkb{96.98\tpm{0.08}},37.08\tpm{0.46},75.56\tpm{0.17}
Var-Monomial,85.46\tpm{5.07},\rkc{77.25\tpm{0.67}},88.38\tpm{0.35},\rkc{37.22\tpm{0.78}},35.21\tpm{2.27},\rka{40.99\tpm{1.84}},36.51\tpm{0.87},65.67\tpm{2.13},42.75\tpm{0.53},79.48\tpm{3.62},78.59\tpm{0.69},\rkc{96.97\tpm{0.07}},36.32\tpm{1.60},78.95\tpm{2.89}
Horner,87.49\tpm{0.91},75.42\tpm{2.38},\rkc{89.69\tpm{0.62}},35.20\tpm{1.14},35.39\tpm{4.46},32.21\tpm{2.17},37.76\tpm{0.70},\rka{67.39\tpm{1.53}},41.67\tpm{0.91},\rkb{85.48\tpm{0.42}},78.24\tpm{0.62},96.96\tpm{0.03},\rkc{37.32\tpm{0.44}},80.17\tpm{3.48}
Chebyshev,73.81\tpm{1.26},71.40\tpm{0.54},88.53\tpm{0.52},35.90\tpm{0.33},35.58\tpm{3.19},37.09\tpm{2.49},37.87\tpm{0.76},64.92\tpm{0.57},40.17\tpm{2.03},75.28\tpm{0.52},78.24\tpm{0.62},96.96\tpm{0.03},37.24\tpm{0.46},75.54\tpm{0.41}
ChebInterp,82.13\tpm{4.68},76.49\tpm{0.55},\rka{90.03\tpm{0.41}},32.24\tpm{3.37},\rka{40.82\tpm{1.17}},36.71\tpm{1.41},31.93\tpm{7.55},65.63\tpm{0.81},\rka{45.03\tpm{0.40}},81.37\tpm{1.08},\rka{79.13\tpm{0.67}},\rka{97.03\tpm{0.07}},31.40\tpm{4.18},73.72\tpm{15.51}


In [34]:
for row in acc_str.index:
    print("\t& " + row + " & " + " & ".join(acc_str.loc[row].values) + " \\\\")
print()

	& Identity & 75.17\tpm{1.36} & 72.93\tpm{0.35} & 87.81\tpm{0.37} & 35.46\tpm{0.17} & 30.52\tpm{1.81} & 32.13\tpm{4.23} & 37.02\tpm{0.50} & 65.30\tpm{0.64} & 43.29\tpm{1.21} & 80.53\tpm{0.87} & 78.32\tpm{0.58} & 96.93\tpm{0.17} & 36.91\tpm{0.91} & 74.61\tpm{0.50} \\
	& Impulse & 86.20\tpm{1.55} & 75.27\tpm{0.26} & 83.12\tpm{0.83} & 23.49\tpm{3.96} & 36.52\tpm{3.93} & 34.83\tpm{1.50} & 25.44\tpm{0.72} & 28.22\tpm{0.80} & 43.34\tpm{0.51} & 80.20\tpm{0.70} & 78.73\tpm{0.59} & 96.96\tpm{0.03} & 23.71\tpm{3.60} & 56.61\tpm{0.70} \\
	& Monomial & \rkb{88.85\tpm{1.19}} & 77.00\tpm{0.98} & 89.37\tpm{0.34} & \rkb{37.41\tpm{0.63}} & 32.40\tpm{2.53} & 34.99\tpm{3.12} & 28.18\tpm{8.12} & 64.85\tpm{0.89} & 41.64\tpm{0.50} & 77.28\tpm{5.05} & 78.70\tpm{0.90} & 96.97\tpm{0.06} & 36.45\tpm{0.86} & 75.85\tpm{0.30} \\
	& PPR & \rka{89.09\tpm{1.82}} & 77.10\tpm{0.46} & 88.94\tpm{0.52} & 36.62\tpm{0.44} & 35.21\tpm{2.34} & 35.89\tpm{3.94} & 37.24\tpm{0.84} & 65.48\tpm{1.26} & \rkc{43.97\tpm{0.81}} & 80.53

## Latex - Eff

In [108]:
mlst = ["time_learn", "time_eval", "mem_cuda_train"]
eff_str = pd.DataFrame(index=flst, columns=pd.MultiIndex.from_product([dlst, mlst]))

for row in eff_str.index:
    for data, met in eff_str.columns:
        mean = df.loc[(df['name'] == row) & (df['data'] == data), met+'_mean'].values
        std = df.loc[(df['name'] == row) & (df['data'] == data), met+'_std'].values
        if len(mean) > 0:
            eff_str.loc[row, (data, met)] = f"{mean[0]:.2f}"

for data,met in eff_str.columns:
    mean = df.loc[df['data'] == data, ['name', met+'_mean']].set_index('name')[met+'_mean'].astype(float)
    std  = df.loc[df['data'] == data, ['name', met+'_std']].set_index('name')[met+'_std'].astype(float)
    rank = calc_ranks(mean, std, True, ['Identity'])
    for i, rki in enumerate(rk):
        idx = rank[rank == i+1].index
        eff_str.loc[idx, (data, met)] = f'\\rk{rki}' + '{' + eff_str.loc[idx, (data, met)] + '}'
    eff_str.loc[rank.isna(), (data, met)] = '\\rko{' + eff_str.loc[rank.isna(), (data, met)].astype(str) + '}'

eff_str

cora                               citeseer              \
              time_learn   time_eval mem_cuda_train  time_learn   time_eval   
Identity      \rko{3.45}  \rko{0.30}     \rko{0.04}  \rko{3.95}  \rko{0.40}   
Impulse       \rka{6.07}        1.37     \rka{0.06}        7.56  \rka{1.10}   
Monomial      \rka{6.10}  \rka{1.07}     \rka{0.06}  \rka{6.45}  \rkc{1.20}   
PPR           \rka{6.75}  \rkb{1.23}     \rka{0.06}  \rka{6.60}  \rkc{1.20}   
HK            \rka{7.10}        1.33     \rka{0.06}  \rka{6.80}  \rkc{1.23}   
Gaussian      \rka{7.08}        1.40     \rka{0.06}  \rka{6.18}  \rkc{1.30}   
Var-Monomial  \rka{7.72}  \rkb{1.13}     \rka{0.06}  \rka{6.04}  \rka{1.00}   
Horner             11.84        1.90           0.07        8.25        1.97   
Chebyshev           8.85        1.60           0.07       12.53        1.93   
ChebInterp         36.65        6.47           0.07       36.35        6.60   
Clenhaw            13.33        1.97           0.07       13.19        2.27   
Bernstein          31.65        7.63           0.21       31.59        8.33   
AdaGNN              8.95        1.47           0.07        9.45        1.53   
ACMGNN             37.41        6.97           0.17       41.03        7.33   
FAGNN              12.84        2.70           0.07        9.77        2.17   
G$^2$CN            16.84        3.13           0.10       16.81        2.60   
GNN-LF/HF          10.56        2.03           0.07        8.47        1.83   
FiGURe             39.90        9.23           0.26       36.28        9.60   

                                  pubmed                             \
             mem_cuda_train   time_learn   time_eval mem_cuda_train   
Identity         \rko{0.08}   \rko{3.85}  \rko{0.43}     \rko{0.10}   
Impulse          \rka{0.10}  \rka{10.84}        2.17     \rka{0.21}   
Monomial         \rka{0.10}  \rka{10.77}  \rkb{2.03}     \rka{0.21}   
PPR              \rka{0.10}  \rka{11.16}        2.30           0.22   
HK               \rka{0.10}  \rka{10.46}  \rkb{2.13}     \rka{0.21}   
Gaussian               0.10  \rka{10.67}  \rka{1.83}           0.21   
Var-Monomial     \rka{0.10}  \rka{11.26}        2.23           0.22   
Horner                 0.12        16.28        5.10           0.34   
Chebyshev              0.11        16.54        4.07           0.33   
ChebInterp             0.12        49.25        7.80           0.34   
Clenhaw                0.12        17.95        5.23           0.34   
Bernstein              0.29        43.52        8.37           1.40   
AdaGNN                 0.11        12.42        3.17           0.31   
ACMGNN                 0.24        37.95        8.90           1.03   
FAGNN                  0.12        20.64        4.07           0.33   
G$^2$CN                0.15        32.22        5.03           0.55   
GNN-LF/HF              0.11        19.91        4.27           0.33   
FiGURe                 0.35        72.08       19.70           1.72   

                  flickr  ...       tolokers    questions              \
              time_learn  ... mem_cuda_train   time_learn   time_eval   
Identity      \rko{3.91}  ...     \rko{0.07}   \rko{4.71}  \rko{2.20}   
Impulse       \rka{9.11}  ...     \rka{0.14}  \rka{22.76}  \rkc{6.10}   
Monomial      \rka{8.77}  ...           0.14  \rka{23.27}  \rka{5.97}   
PPR                10.05  ...     \rka{0.14}  \rka{22.76}  \rkc{6.30}   
HK            \rkc{9.66}  ...     \rka{0.14}  \rka{22.11}  \rkc{6.17}   
Gaussian      \rkc{9.59}  ...           0.17  \rka{23.27}  \rka{6.07}   
Var-Monomial  \rkc{9.28}  ...           0.14  \rka{23.23}        6.70   
Horner             11.61  ...           0.23        30.09       11.93   
Chebyshev          13.47  ...           0.23        30.07        9.43   
ChebInterp         35.70  ...           0.24        62.52       13.80   
Clenhaw            12.44  ...           0.23        31.69       12.97   
Bernstein          38.74  ...           0.87       125.98       12.73

In [109]:
dlst_eff = ["flickr", "reddit", "tolokers", "penn94"]
for row in eff_str.index:
    print("\t& " + row + " & " + " & ".join(eff_str.loc[row, dlst_eff].values) + " \\\\")
print()

	& Identity & \rko{3.91} & \rko{0.50} & \rko{0.06} & \rko{3.45} & \rko{0.50} & \rko{0.06} & \rko{3.60} & \rko{0.60} & \rko{0.07} & \rko{9.03} & \rko{1.93} & \rko{0.93} \\
	& Impulse & \rka{9.11} & \rkb{1.10} & 0.11 & \rka{8.34} & \rka{1.00} & \rkb{0.11} & \rka{21.71} & \rkb{1.67} & \rka{0.14} & \rka{57.11} & \rkb{6.87} & \rka{1.19} \\
	& Monomial & \rka{8.77} & \rkb{1.03} & \rka{0.10} & \rka{8.93} & 1.23 & \rkb{0.11} & \rka{21.94} & \rkb{1.50} & 0.14 & \rka{56.81} & \rkb{6.83} & \rka{1.19} \\
	& PPR & 10.05 & \rka{0.97} & \rka{0.10} & 9.66 & 1.23 & \rkb{0.11} & \rka{22.81} & \rkb{1.57} & \rka{0.14} & \rka{57.21} & \rkb{7.13} & \rka{1.19} \\
	& HK & \rkc{9.66} & 1.20 & 0.11 & \rka{9.17} & \rkb{1.03} & \rkb{0.11} & \rka{21.93} & \rkb{1.50} & \rka{0.14} & \rka{57.62} & \rkb{6.87} & \rka{1.19} \\
	& Gaussian & \rkc{9.59} & 1.50 & 0.11 & 9.87 & 1.50 & 0.11 & \rka{22.47} & \rka{1.33} & 0.17 & \rka{56.88} & \rka{6.10} & 1.26 \\
	& Var-Monomial & \rkc{9.28} & 1.27 & \rkc{0.11} & 10.55 & \rkc{1